In [36]:
import procedural
import json
import os

master_path = "/home/kongsgaard/git/procedural-py/"
secret_path = os.path.join(master_path, "secrets")
data_path = os.path.join(master_path, "examples", "data")


In [37]:
user, password, url = procedural.utils.get_secrets(secret_path)
client = procedural.Client(url, user, password)
print(f"CLIENT: {client}")


CLIENT: http://localhost:8001 - Authenticated: Yes


In [38]:
project_name = "Jupyter Project"
project_number = 99
project = client.get_or_create("api/project/", {"name": project_name, "number": project_number})
project_id = project.get("uid")

print(f"PROJECT: {project_id[:8]}")
print(json.dumps(project, indent=4))

PROJECT: 733958e7
{
    "uid": "733958e7-cb8e-4c0b-94da-3d8d8e71b4de",
    "name": "Jupyter Project",
    "number": 99,
    "cluster": "2a6812b0-9b95-4336-8819-26889a677b18",
    "path": "/mnt/cluster/projects/99_jupyter-project",
    "path_type": "name",
    "company": "30eb91f9-d82c-4d1a-9370-9461ac531b92"
}


In [39]:
task_name = "Jupyter Serial VWT"
parent_task = client.get_or_create(
    f"api/project/{project.get('uid')}/task/",
    {"name": task_name},
    {"config": {
        "task_type": "parent",
        "case_dir": "foam"
    }
    })
parent_id = parent_task.get("uid")

print(f"PARENT TASK: {parent_id[:8]}")
print(json.dumps(parent_task, indent=4))


PARENT TASK: 4c16267c
{
    "uid": "4c16267c-6d12-4615-a8e7-bed64fd351b5",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-26T09:19:35.625763",
    "config": {
        "case_dir": "foam",
        "task_type": "parent"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-serial-vwt_59cbac84",
    "parent": null,
    "meta_json": {},
    "project": "733958e7-cb8e-4c0b-94da-3d8d8e71b4de",
    "name": "Jupyter Serial VWT",
    "dependent_on": null
}


In [40]:
stl_path = os.path.join(data_path, "mesh_vwt", "cfdGeom.stl")
file_upload = client.update(f"api/task/{parent_id}/file/foam/constant/triSurface/cfdGeom.stl/",
                            files=open(stl_path, 'r').read())
print(f"UPLOADED FILE: {file_upload.get('file')}")

UPLOADED FILE: foam/constant/triSurface/cfdGeom.stl


In [41]:
write_mesh_path = os.path.join(data_path, "mesh_vwt", "write_mesh.json")
with open(write_mesh_path, 'r') as f:
    mesh_config = json.load(f)

payload = {"name": "Actions", "config": mesh_config, "parent": parent_id}
action_task = client.create(f"/api/project/{project_id}/task/", payload)
action_id = action_task.get("uid")

write_solution_path = os.path.join(data_path, "mesh_vwt", "write_solution.json")
with open(write_solution_path, 'r') as f:
    solution_config = json.load(f)

payload = {"name": "Actions", "status": "pending", "config": solution_config}
client.update(f"/api/project/{project_id}/task/{action_id}/", payload)

print(f"ACTION TASK: {action_id[:8]}")
print(json.dumps(action_task, indent=4))

configs = client.get(f"/api/task/{action_id}/config/")
print(f"TASK CONFIGS FOR ACTION TASK: {action_id[:8]}")
print(json.dumps(configs, indent=4))

ACTION TASK: d1639ff1
{
    "uid": "d1639ff1-9e22-44c4-a99f-01890788e11f",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-26T09:19:35.743338",
    "config": {
        "cmd": "cfd.io.tasks.write_mesh",
        "case_dir": "foam",
        "base_mesh": {
            "type": "simpleBox",
            "cell_size": 2,
            "parameters": {
                "z0": "True",
                "square": "True"
            },
            "bounding_box": {
                "max": [
                    35,
                    35,
                    24
                ],
                "min": [
                    -35,
                    -35,
                    0
                ]
            }
        },
        "task_type": "magpy",
        "snappyhex_mesh": {
            "surfaces": {
                "_269c2d39-4173-45a4-8f97-a21f3917d174": {
                    "level": {
                        "max": "2",
                        "min": "2"
       

In [42]:
cpus = [2, 1, 1]
payload = {
    "name": "Mesh",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "cpus": cpus,
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "!checkMesh -writeSets vtk"
        ]},
    "parent": parent_id,
    "dependent_on": action_id
}

mesh_task = client.create(f"/api/project/{project_id}/task/", payload)
mesh_id = mesh_task.get("uid")

print(f"MESH TASK: {mesh_id[:8]}")
print(json.dumps(mesh_task, indent=4))


MESH TASK: a4fb381a
{
    "uid": "a4fb381a-681a-4acd-83b8-7a30b1e3265c",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-26T09:19:35.915817",
    "config": {
        "cmd": "pipeline",
        "cpus": [
            2,
            1,
            1
        ],
        "case_dir": "foam",
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "!checkMesh -writeSets vtk"
        ],
        "task_type": "cfd"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-serial-vwt_59cbac84",
    "parent": {
        "uid": "4c16267c-6d12-4615-a8e7-bed64fd351b5",
        "name": "Jupyter Serial VWT"
    },
    "meta_json": {},
    "project": "733958e7-cb8e-4c0b-94da-3d8d8e71b4de",
    "name": "Mesh",
    "dependent_on": {
        "uid": "d1639ff1-9e22-44c4-a99f-01890788e11f",
        "name": "Actions"
    }
}


In [43]:
payload_vwt = {
    "name": "VWT",
    "config": {
        "task_type": "cfd",
        "cmd": "wind_tunnel",
        "commands": [
            [0, 10, 20],
            [30, 40, 50],
            [60, 70, 80]
        ],
        "cpus": cpus,
        "iterations": {
            "init": 100,
            "run": 50
        }
    },
    "parent": parent_id,
    "dependent_on": mesh_id
}

vwt_task = client.create(f"/api/project/{project_id}/task/", payload_vwt)
vwt_id = vwt_task.get("uid")

print(f"VWT TASK: {vwt_id[:8]}")
print(json.dumps(vwt_task, indent=4))

VWT TASK: b158659b
{
    "uid": "b158659b-29f5-4c17-848e-f3b00f076121",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-26T09:19:36.000061",
    "config": {
        "cmd": "wind_tunnel",
        "cpus": [
            2,
            1,
            1
        ],
        "case_dir": "foam",
        "commands": [
            [
                0,
                10,
                20
            ],
            [
                30,
                40,
                50
            ],
            [
                60,
                70,
                80
            ]
        ],
        "task_type": "parent",
        "iterations": {
            "run": 50,
            "init": 100
        }
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-serial-vwt_59cbac84",
    "parent": {
        "uid": "4c16267c-6d12-4615-a8e7-bed64fd351b5",
        "name": "Jupyter Serial VWT"
    },
    "meta_json": {},
    "project": "733958e

In [44]:
payload = {
    "name": "Probing",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "commands": [
            "write_sample_set",
            "postProcess -fields '(U p)' -func internalCloud"
        ],
        "case_dir": "VWT/",
        "cpus": [1, 1, 1],
        "sets": [
            {"name": "set1",
             "points": [
                 [0, 0, 1],
                 [0, 1, 1]
             ]}
        ],
        "fields": ["U", "p"]
    },
    "parent": parent_id,
    "dependent_on": vwt_id
}
probe_task = client.create(f"/api/project/{project_id}/task/", payload)
probe_id = probe_task.get("uid")

print(f"PROBE TASK: {probe_id[:8]}")
print(json.dumps(probe_task, indent=4))


PROBE TASK: f57f926b
{
    "uid": "f57f926b-d46b-4e33-9b7e-7466e74d096b",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-26T09:19:36.219824",
    "config": {
        "cmd": "pipeline",
        "cpus": [
            1,
            1,
            1
        ],
        "sets": [
            {
                "name": "set1",
                "points": [
                    [
                        0,
                        0,
                        1
                    ],
                    [
                        0,
                        1,
                        1
                    ]
                ]
            }
        ],
        "fields": [
            "U",
            "p"
        ],
        "case_dir": "VWT/",
        "commands": [
            "write_sample_set",
            "postProcess -fields '(U p)' -func internalCloud"
        ],
        "task_type": "cfd"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-pro

In [45]:
#client.delete(f"api/project/{project_id}/task/{parent_id}/")
#client.delete(f"api/project/{project_id}/")